In [ ]:
# Bibliotecas
import pandas as pd

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model

In [ ]:
# Lendo dados;
data = pd.read_csv("../Databases/games.csv")

In [ ]:
# Filtros;
data = data[data["NA_Sales"] > 1]
data = data.loc[data["EU_Sales"] > 1]

# Apagando colunas irrelevantes;
data = data.drop(["Other_Sales", "NA_Sales","EU_Sales","JP_Sales", "Developer","Name"], axis=1)

# Apagando linhas com valores faltantes;
data = data.dropna(axis=0)



In [ ]:
# Divisão da base de dados;
X = data.iloc[:,[0, 1, 2, 3, 5, 6, 7, 8, 9]].values
sales_global = data.iloc[:, 4].values

# Transformando categoricas para numerico;
encoder = LabelEncoder()
categorical = [0, 2, 3, 8]
for i in categorical:
    X[:,i] = encoder.fit_transform(X[:,i])

# Transformando pra p formato oneHot
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), categorical)],remainder='passthrough')
X = onehotencorder.fit_transform(X).toarray()

### Criando rede neural

In [ ]:
# Criando camadas;
#--camada de entrada;
input_layer = Input(shape=(61,))
#--camada Ocultas;
hidden_layer_1 = Dense(units=32, activation=Activation("sigmoid"))(input_layer)
hidden_layer_2 = Dense(units=32, activation=Activation("sigmoid"))(hidden_layer_1)
#--camada de saídas;
out_layer_1 = Dense(units=1, activation=Activation("linear"))(hidden_layer_2)

# Criando Regressor;
Regressor = Model(inputs= input_layer,
              outputs=[out_layer_1])
Regressor.compile(optimizer = "adam", loss="mse")

# Treinamento
Regressor.fit(X, [sales_global], epochs=5000, batch_size=100)

In [ ]:
# Previsão apenas para demonstração
pred_global = Regressor.predict(X)